In [69]:
# SOURCE
SOURCE = "https://www.tesladeaths.com"


In [70]:
from requests_html import HTMLSession, Element

session = HTMLSession()
r: session = session.get(SOURCE)


Getting the `table` element that houses the data.



In [71]:
table: Element = r.html.find("#dttable", first=True)
table


<Element 'table' class=('waffle',) cellspacing='0' cellpadding='0' aria-describedby='tblDesc' id='dttable' summary='Crashes involving Tesla deaths, from 2013 to 2023'>

Finding all row headers using `th` tag.



In [72]:
table_headers = table.find("th")[:12]
table_headers = [row.text for row in table_headers]
table_headers


['Case #',
 'Year',
 'Date',
 'Country',
 'State',
 'Description',
 'Deaths',
 'Tesla driver',
 'Tesla occupant',
 'Other vehicle',
 'Cyclists/ Peds',
 'TSLA+cycl / peds']

Collecting all data rows and excluding the last 350 since there is no usable data in there.



In [ ]:
table_data = table.find("tr")[1:]
# The first tr has all the headings


In [74]:
rows = [row.text.split("\n")[:12] for row in table_data]
for row in rows:
    print(row)


['458', '2024', '10/16/2024', 'USA', 'CA', 'Head-on collision', '1', '-', '-', '1', '-', '-']
['457', '2024', '10/15/2024', 'USA', 'CA', 'Tesla crashes into apartment building', '1', '1', '-', '-', '-', '1']
['456', '2024', '10/14/2024', 'Korea', '-', 'Tesla crashes, burns', '1', '1', '-', '-', '-', '1']
['455', '2024', '10/13/2024', 'France', '-', 'Tesla ignites for undetermined reason', '4', '1', '3', '-', '-', '4']
['454', '2024', '10/7/2024', 'Australia', '-', 'Head-on collision on highway', '1', '1', '-', '-', '-', '-']
['453', '2024', '10/5/2024', 'Mexico', '-', 'Tesla tries to pass, collides with van, fire, 3 dead', '3', '-', '-', '3', '-', '-']
['452.1', '2024', '10/2/2024', 'China', '-', 'Speeding Tesla hits three pedestrians', '3', '-', '-', '-', '3', '3']
['452', '2024', '9/27/2024', 'USA', 'IL', 'Tesla tries to pass, collides head on, kills other driver', '1', '-', '-', '1', '-', '-']
['451.1', '2024', '9/24/2024', 'USA', 'IL', 'Tesla crashes into guardrail', '1', '1', '-',

There were some challenges arranging the data in the right way.

-  ~~The urls in the table are truncated, which means simply finding the text would not suffice.~~ Changed approach to `tr` instead of `td`. Extracted the first 12 columns which is what we really need.

~~-   There are more than **one** URL per row. There are numbers in the table are _hyperlinked_ which means there are more URLs than are rows, which makes it difficult to simply find all the `a` tags and plug them into the table at the right index while looping through all the elements in `table_data`. Speaking of which...~~

~~-   The collection of elements in `table_data` is simply a dump of the table; not by row. There are 23 columns in the table therefore, for each 23 elements found from the beginning is one row.~~



Mandatory conversion to DataFrame 😅



In [75]:
import pandas as pd

df = pd.DataFrame(rows, columns=table_headers)
# Converting necessary columns from str to int values
int_value_columns = df.columns[6:12]
df[int_value_columns] = (
    df[int_value_columns].apply(pd.to_numeric, errors="coerce").fillna(0).astype("int")
)

df.dtypes


Case #              object
Year                object
Date                object
Country             object
State               object
Description         object
Deaths               int64
Tesla driver         int64
Tesla occupant       int64
Other vehicle        int64
Cyclists/ Peds       int64
TSLA+cycl / peds     int64
dtype: object

In [76]:
df.head()


,Case #,Year,Date,Country,State,Description,Deaths,Tesla driver,Tesla occupant,Other vehicle,Cyclists/ Peds,TSLA+cycl / peds
0,458,2024,10/16/2024,USA,CA,Head-on collision,1,0,0,1,0,0
1,457,2024,10/15/2024,USA,CA,Tesla crashes into apartment building,1,1,0,0,0,1
2,456,2024,10/14/2024,Korea,-,"Tesla crashes, burns",1,1,0,0,0,1
3,455,2024,10/13/2024,France,-,Tesla ignites for undetermined reason,4,1,3,0,0,4
4,454,2024,10/7/2024,Australia,-,Head-on collision on highway,1,1,0,0,0,0


In [77]:
df.tail(15)


,Case #,Year,Date,Country,State,Description,Deaths,Tesla driver,Tesla occupant,Other vehicle,Cyclists/ Peds,TSLA+cycl / peds
487,4,2014,7/4/2014,USA,CA,Thief crashes stolen Tesla,1,1,0,0,0,1
488,3,2014,7/4/2014,USA,CA,Tesla rear ends stopped car,3,0,0,3,0,0
489,2,2013,11/2/2013,USA,CA,Tesla kills cyclist,1,0,0,0,1,1
490,1,2013,4/2/2013,USA,CA,Tesla veers into opposite lane,2,0,0,2,0,0
491,,None,None,None,None,None,0,0,0,0,0,0
492,International,140,45,19,53,18,95,8,3,0,0,0
493,USA,474,165,81,170,42,266,47,40,48,0,0
494,Total,614,210,100,223,60,361,55,40,51,0,0
495,,None,None,None,None,None,0,0,0,0,0,0
496,116,143,119,None,None,None,0,0,0,0,0,0


In [78]:
cutoff_point = df[df["Case #"] == "1"].index.values[0] + 1


In [79]:
df = df.iloc[:cutoff_point]
df


,Case #,Year,Date,Country,State,Description,Deaths,Tesla driver,Tesla occupant,Other vehicle,Cyclists/ Peds,TSLA+cycl / peds
0,458,2024,10/16/2024,USA,CA,Head-on collision,1,0,0,1,0,0
1,457,2024,10/15/2024,USA,CA,Tesla crashes into apartment building,1,1,0,0,0,1
2,456,2024,10/14/2024,Korea,-,"Tesla crashes, burns",1,1,0,0,0,1
3,455,2024,10/13/2024,France,-,Tesla ignites for undetermined reason,4,1,3,0,0,4
4,454,2024,10/7/2024,Australia,-,Head-on collision on highway,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
486,5,2014,7/14/2014,USA,CA,Tesla kills motorcyclist,1,0,0,1,0,0
487,4,2014,7/4/2014,USA,CA,Thief crashes stolen Tesla,1,1,0,0,0,1
488,3,2014,7/4/2014,USA,CA,Tesla rear ends stopped car,3,0,0,3,0,0
489,2,2013,11/2/2013,USA,CA,Tesla kills cyclist,1,0,0,0,1,1


Played around with the new library `Polars` 🐻‍❄️ which is supposed to be [`faaaast`](https://www.youtube.com/shorts/6E7ZGCfruaw). It is indeed, or should be in theory. `Polars` store data in DataFrames in _columnar_ format as opposed to the classical row format _Pandas_ 🐼 uses.



In [1]:
# import polars as pl

# pl_rows = pl.DataFrame(rows)
# pl_rows


In [81]:
df.dtypes


Case #              object
Year                object
Date                object
Country             object
State               object
Description         object
Deaths               int64
Tesla driver         int64
Tesla occupant       int64
Other vehicle        int64
Cyclists/ Peds       int64
TSLA+cycl / peds     int64
dtype: object

Converting from `mm/dd/yyyy` to `dd/mm/yyyy`. It is possible to convert them into `datetime` objects.



In [82]:
df["Date"]


0      10/16/2024
1      10/15/2024
2      10/14/2024
3      10/13/2024
4       10/7/2024
          ...    
486     7/14/2014
487      7/4/2014
488      7/4/2014
489     11/2/2013
490      4/2/2013
Name: Date, Length: 491, dtype: object

In [83]:
from datetime import datetime
import logging

logging.basicConfig(format="%(asctime)s %(levelname)s: %(message)s", level=logging.INFO)


def convert_datestring(date_string):
    try:
        date_object = datetime.strptime(date_string, "%m/%d/%Y")
        return date_object.strftime("%Y-%m-%d")
    except ValueError as e:
        logging.warning(f"Could not convert {date_string} into datetime object: {e}")
        logging.info("Assigning a random date.")
        date_parts = date_string.split("/")
        for index, date_part in enumerate(date_parts):
            try:
                int(date_part)
            except ValueError:
                date_parts[index] = "12"
                return convert_datestring("/".join(date_parts))


df["Date"] = df["Date"].apply(convert_datestring)


2024-11-01 20:49:34,029 WARNING:Could not convert 9/-/2022 into datetime object: time data '9/-/2022' does not match format '%m/%d/%Y'
2024-11-01 20:49:34,031 INFO:Assigning a random date.
2024-11-01 20:49:34,033 WARNING:Could not convert 6/-/2022 into datetime object: time data '6/-/2022' does not match format '%m/%d/%Y'
2024-11-01 20:49:34,034 INFO:Assigning a random date.
2024-11-01 20:49:34,037 WARNING:Could not convert 1/-/2022 into datetime object: time data '1/-/2022' does not match format '%m/%d/%Y'
2024-11-01 20:49:34,038 INFO:Assigning a random date.
2024-11-01 20:49:34,041 WARNING:Could not convert 8/-/2019 into datetime object: time data '8/-/2019' does not match format '%m/%d/%Y'
2024-11-01 20:49:34,042 INFO:Assigning a random date.


In [84]:
df["Date"]


0      2024-10-16
1      2024-10-15
2      2024-10-14
3      2024-10-13
4      2024-10-07
          ...    
486    2014-07-14
487    2014-07-04
488    2014-07-04
489    2013-11-02
490    2013-04-02
Name: Date, Length: 491, dtype: object

Now let's convert the dates into a datetime object



In [85]:
df["Date"] = pd.to_datetime(df["Date"])
df["Date"]


0     2024-10-16
1     2024-10-15
2     2024-10-14
3     2024-10-13
4     2024-10-07
         ...    
486   2014-07-14
487   2014-07-04
488   2014-07-04
489   2013-11-02
490   2013-04-02
Name: Date, Length: 491, dtype: datetime64[ns]

`Holland` and `Netherlands` appear in the list of countries, although they are the same! Replacing the former with the latter.



In [86]:
df["Country"].replace({"Holland": "Netherlands"}, inplace=True)


Finally! Write the data out to a `.csv` file.



In [87]:
df.to_csv("./data.csv", index=None)


`dtypes` change when reading from `csv`, as it tries to infer the data type for each column. For example, the year converts to `int64`. It is possible to change the `dtype` into something else by passing the `dtype` argument in `read_csv` using a `key-value` of a column name and desired `dtype`.



In [88]:
df = pd.read_csv("./data.csv")
df.dtypes


Case #              object
Year                 int64
Date                object
Country             object
State               object
Description         object
Deaths               int64
Tesla driver         int64
Tesla occupant       int64
Other vehicle        int64
Cyclists/ Peds       int64
TSLA+cycl / peds     int64
dtype: object

In [148]:
df = pd.read_csv("./data.csv", dtype={"Case #": str, "Year": str, "Date": str})
df.dtypes


Case #              object
Year                object
Date                object
Country             object
State               object
Description         object
Deaths               int64
Tesla driver         int64
Tesla occupant       int64
Other vehicle        int64
Cyclists/ Peds       int64
TSLA+cycl / peds     int64
dtype: object

In [89]:
# Last updated on
import re

pattern = re.compile("\d{4}-\d{2}-\d{2}")
html = r.html.find("em")

last_updated_on = []
for em in html:
    match = pattern.search(em.text)
    if match:
        last_updated_on.append(match.group())

last_updated_on


['2024-10-20']